In [3]:
import os
import torch
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

In [6]:
from graph_bridges.models.backward_rates.backward_rate import GaussianTargetRateImageX0PredEMA
from graph_bridges.data.dataloaders import DoucetTargetData
from graph_bridges.models.samplers.sampling import ReferenceProcess
from graph_bridges.models.reference_process.ctdd_reference import ReferenceProcess
from graph_bridges.models.losses.ctdd_losses import GenericAux
from graph_bridges.models.schedulers.scheduling_ctdd import CTDDScheduler

from graph_bridges.models.pipelines.pipelines_utils import create_pipelines
from graph_bridges.models.schedulers.scheduling_utils import create_scheduler
from graph_bridges.models.backward_rates.backward_rate_utils import create_model
from graph_bridges.models.reference_process.reference_process_utils import create_reference
from graph_bridges.data.dataloaders_utils import create_dataloader
from graph_bridges.models.losses.loss_utils import create_loss

from graph_bridges.configs.graphs.lobster.config_base import BridgeConfig

In [7]:
config = BridgeConfig()
device = torch.device(config.device)



In [8]:
#=================================================================
# CREATE OBJECTS FROM CONFIGURATION

data_dataloader: DoucetTargetData
model : GaussianTargetRateImageX0PredEMA
reference_process: ReferenceProcess
loss : GenericAux
scheduler:CTDDScheduler

data_dataloader = create_dataloader(config,device)
model = create_model(config,device)
reference_process = create_reference(config,device)
loss = create_loss(config,device)
scheduler = create_scheduler(config,device)

Scheduler


In [10]:
#=================================================================
sample_ = data_dataloader.sample(config.number_of_paths, device)
minibatch = sample_.unsqueeze(1).unsqueeze(1)


In [12]:
# TIME ===========================================================
#ts = torch.rand((minibatch.shape[0],), device=device) * (1.0 - config.loss.min_time) + config.loss.min_time
B = minibatch.shape[0]
ts = torch.rand((B,), device=device) * (1.0 - config.loss.min_time) + config.loss.min_time
#==========

x_t, x_tilde, qt0, rate = scheduler.add_noise(minibatch,reference_process,ts,device,return_dict=False)

x_logits,p0t_reg,p0t_sig,reg_x = model.forward(minibatch,ts,x_tilde)

In [14]:
scheduler_noise_output = scheduler.add_noise(minibatch,reference_process,ts,device,return_dict=True)

model_forward_output = model.forward(minibatch,ts,x_tilde,return_dict=True)

In [15]:
scheduler_noise_output.keys()

odict_keys(['x_t', 'x_tilde', 'qt0', 'rate'])

In [16]:
model_forward_output.keys()

odict_keys(['x_logits', 'p0t_reg', 'p0t_sig', 'reg_x'])

In [ ]:
loss_ = loss.calc_loss(minibatch,x_tilde,qt0,rate,x_logits,reg_x,p0t_sig,p0t_reg,device)